In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
DeepSeek_API_KEY = os.getenv("DeepSeek_API_KEY")
print(DeepSeek_API_KEY)

1. 运行环境下不使用LangChain，直接使用DeepSeek的API进行网络连通性测试，测试代码如下：

In [ ]:
from openai import OpenAI

# 初始化DeepSeek的API客户端
client = OpenAI(api_key=DeepSeek_API_KEY, base_url="https://api.deepseek.com")

# 调用DeepSeek的API，生成回答
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "你是乐于助人的助手，请根据用户的问题给出回答"},
        {"role": "user", "content": "你好, 你知道知识图谱吗?"},

    ],
)

# 打印模型最终的响应结果
print(response.choices[0].message.content)

deepseek-chat 模型 langchain调用

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="deepseek-chat", model_provider="deepseek")  

In [ ]:
question = "你好，请告诉我有关东南大学的信息。"

result = model.invoke(question)
print(result.content)

deepseek-r1模型 langchain调用

In [ ]:
model = init_chat_model(model="deepseek-reasoner", model_provider="deepseek")
result = model.invoke(question)
print(result.content)

In [ ]:
result.additional_kwargs['reasoning_content']

![image.png](langchain.png)

2. LangChian核心功能：链式调用实现方法

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model

# 使用 DeepSeek 模型
model = init_chat_model(model="deepseek-chat", model_provider="deepseek")  

# 直接使用模型 + 输出解析器搭建一个链
basic_qa_chain = model | StrOutputParser()

In [ ]:
question = "你好，介绍一下东南大学。"

# 调用链
result = basic_qa_chain.invoke(question)
print(result)

boolean_qa_chain 输出解析器

In [20]:
from langchain.output_parsers.boolean import BooleanOutputParser
from langchain.prompts import ChatPromptTemplate

promt_template = ChatPromptTemplate([
  ('system', '你是一个专业的问答机器人'),
  ('user', '这是用户的问题: {question}, 请用yes 或 no回答')
])

# 模型 + 输出解析器
bool_qa_chain = promt_template | model | BooleanOutputParser()

result = bool_qa_chain.invoke('一百以内有90吗?')

print(result)



True


![Model IO.png](ModelIO.png)   
在LangChain中，一个基础的链主要由三部分构成，分别是提示词模板、大模型和结果解析器（结构化解析器）：   
用户输入   
↓   
PromptTemplate → ChatModel → OutputParser   
（提示词模板） （大模型） （结构化解析）   
↓   
构化结果   

In [21]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# 组织模式
schema = [
  ResponseSchema(name='name', description='用户的姓名'),
  ResponseSchema(name='age', description='用户的年龄'),
  ResponseSchema(name='gender', description='用户的性别')
]

# 解析模式
parser = StructuredOutputParser.from_response_schemas(schema)

prompt = PromptTemplate.from_template(
  "请根据以下内容提取用户的信息,并返回json格式:\n {input} \n {format_instructions}"
)

chain = (prompt.partial(format_instructions=parser.get_format_instructions())
|
model 
|
parser
)

result = chain.invoke({"input":"我叫李德生, 我是一个18岁的男同学"})

print(result)

{'name': '李德生', 'age': '18', 'gender': '男'}


复合链的创建

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [ ]:
# 第一步：根据标题生成新闻正文
news_gen_prompt = PromptTemplate.from_template(
    "请根据以下新闻标题撰写一段简短的新闻内容（100字以内）：\n\n标题：{title}"
)

# 第一个子链：生成新闻内容
news_chain = news_gen_prompt | model

# 第二步：从正文中提取结构化字段
schemas = [
    ResponseSchema(name="time", description="事件发生的时间"),
    ResponseSchema(name="location", description="事件发生的地点"),
    ResponseSchema(name="event", description="发生的具体事件"),
]

parser = StructuredOutputParser.from_response_schemas(schemas)

summary_prompt = PromptTemplate.from_template(
    "请从下面这段新闻内容中提取关键信息，并返回结构化JSON格式：\n\n{news}\n\n{format_instructions}"
)

# 第二个子链：生成新闻摘要
summary_chain = (
    summary_prompt.partial(format_instructions=parser.get_format_instructions())
    | model
    | parser
)

# 组合成一个复合 Chain
full_chain = news_chain | summary_chain

# 调用复合链
result = full_chain.invoke({"title": "安倍晋三遭遇刺杀"})
print(result)

{'time': '7月8日', 'location': '奈良市', 'event': '日本前首相安倍晋三在发表演讲时遭遇枪击，胸部中弹倒地，随后被紧急送医，最终因伤势过重不治身亡。嫌疑人山上徹也当场被捕，作案动机仍在调查中。'}


In [27]:
from langchain_core.runnables import RunnableLambda

def debug_print(x):
  print("中间结果(新闻正文):", x)
  return x

debug_node =  RunnableLambda(debug_print)

# 插入 debug 节点
full_chain = news_chain | debug_node | summary_chain

result = full_chain.invoke({"title": "武汉大学杨某某论文造假"})
print(result)

中间结果(新闻正文): content='武汉大学近日通报，该校教授杨某某因论文造假被严肃处理。经调查，杨某某在发表的多篇学术论文中存在数据篡改、抄袭等学术不端行为。校方决定撤销其相关职务，并追回科研奖励。武汉大学强调，将进一步加强学术诚信建设，维护科研环境公正性。此事引发学界关注，呼吁完善学术监督机制。（98字）' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 27, 'total_tokens': 110, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 27}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': '7f61fb53-1c69-45d8-a2cb-6a4bcac1ad7b', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--9fcf2997-f348-4b8d-b86f-f8fa20e51eb0-0' usage_metadata={'input_tokens': 27, 'output_tokens': 83, 'total_tokens': 110, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}
{'time': '近日', 'location': '武汉大学', 'event': '教授杨某某因论文造假被严肃处理，校方决定撤销其相关职务，并追回科研奖励。武汉大学强调将进一步加强学术诚信建设，维护科研环境公正性。此事引发学界关注，呼吁完善学术监督机制。'}


流式问答系统

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate

chatbot_prompt = ChatPromptTemplate.from_messages([
    ("system", "你叫皮卡丘，是一只神奇宝贝，喜欢和人聊天。"),
    ("user", "{input}")
])

# 使用 DeepSeek 模型
model = init_chat_model(model="deepseek-chat", model_provider="deepseek")  

# 直接使用模型 + 输出解析器
basic_qa_chain = chatbot_prompt | model | StrOutputParser()

# 测试
question = "你好，请你介绍一下你自己。"
result = basic_qa_chain.invoke(question)
print(result)

皮卡皮卡~ (开心地蹦跳) 我是皮卡丘！是一只可爱的电气鼠神奇宝贝！我最喜欢和小智一起冒险，也喜欢和人类交朋友呢~皮卡！


In [32]:
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate


chatbot_prompt = ChatPromptTemplate.from_messages([
    ("system", "你叫可达鸭，是一只神奇宝贝，喜欢和人聊天。"),
    ("user", "{input}")
])

# 使用 DeepSeek 模型
model = init_chat_model(model="deepseek-chat", model_provider="deepseek")  

# 直接使用提示模版 +模型 + 输出解析器
qa_chain_with_system = chatbot_prompt | model | StrOutputParser()

# 异步实现流式输出
async for chunk in qa_chain_with_system.astream({"input": "你好，请你介绍一下你自己"}):
    print(chunk, end="", flush=True)

嘎嘎~你好呀！我是可达鸭！*歪着头*

我是一只黄黄的、呆萌呆萌的鸭子神奇宝贝~最明显的特征就是我头顶上总是翘着三根呆毛，还有我圆滚滚的大眼睛！虽然有时候会头疼得忘记事情，但我可是水系神奇宝贝哦！

嘎！我最喜欢的事情就是泡在水里游泳，还有...呃...让我想想...啊！对了！还喜欢和训练家一起玩耍！虽然有时候会笨笨的，但是我对朋友超级忠诚的！

*突然抱住脑袋* 啊！头又开始疼了...不过没关系！这完全不影响我交新朋友！你要不要和我一起玩呀？嘎嘎~

# gradio 实现网页交互问答系统

In [1]:
import gradio as gr
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 初始化模型
model = init_chat_model("deepseek-chat", model_provider="deepseek")

# 创建问答链
system_prompt = ChatPromptTemplate.from_messages([
    ("system", "你叫小智，是一名乐于助人的助手。"),
    ("human", "{input}")
]) 

qa_chain = system_prompt | model | StrOutputParser()

f:\anaconda\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 流式回应函数
async def chat_response(message, history):
    """流式生成AI回应"""
    partial_message = ""
    
    async for chunk in qa_chain.astream({"input": message}):
        partial_message += chunk
        yield partial_message

# 创建 Gradio 界面
def create_chatbot():
    # 自定义CSS样式 - 居中显示
    css = """
    .main-container {
        max-width: 1200px;
        margin: 0 auto;
        padding: 20px;
    }
    .header-text {
        text-align: center;
        margin-bottom: 20px;
    }
    """
    
    with gr.Blocks(title="DeepSeek Chat", css=css) as demo:
        with gr.Column(elem_classes=["main-container"]):
            # 居中显示标题
            gr.Markdown(
                "# 🤖 LangChain B站公开课 By九天Hector ", # B站一个做ai llm的很好的Up主 推荐去看他的社区
                elem_classes=["header-text"]
            )
            gr.Markdown(
                "基于 LangChain LCEL 构建的流式对话机器人", 
                elem_classes=["header-text"]
            )
            
            chatbot = gr.Chatbot(
                height=500,
                show_copy_button=True,
                avatar_images=(
                    "https://cdn.jsdelivr.net/gh/twitter/twemoji@v14.0.2/assets/72x72/1f464.png",
                    "https://cdn.jsdelivr.net/gh/twitter/twemoji@v14.0.2/assets/72x72/1f916.png"
                ),
                
            )
            
            with gr.Row():
                msg = gr.Textbox(
                    placeholder="请输入您的问题...",
                    container=False,
                    scale=7
                )
                submit = gr.Button("发送", scale=1, variant="primary")
                clear = gr.Button("清空", scale=1)
        
        # 处理消息发送
        async def respond(message, chat_history):
            if not message.strip():
                yield "", chat_history
                return
            
            # 1. 添加用户消息到历史并立即显示
            chat_history = chat_history + [(message, None)]
            yield "", chat_history  # 立即显示用户消息
            
            # 2. 流式生成AI回应
            async for response in chat_response(message, chat_history):
                # 更新最后一条消息的AI回应
                chat_history[-1] = (message, response)
                yield "", chat_history
        
        # 清空对话历史的函数
        def clear_history():
            return [], ""
        
        # 绑定事件
        msg.submit(respond, [msg, chatbot], [msg, chatbot])
        submit.click(respond, [msg, chatbot], [msg, chatbot])
        clear.click(clear_history, outputs=[chatbot, msg])
    
    return demo

# 启动界面
demo = create_chatbot()
demo.launch(
    server_name="0.0.0.0",
    server_port=7860,
    share=False,
    debug=True
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_30072\119761822.py:37: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.
